# Getting Started with the Nemo Retriever "Retriever" microservice
## Demonstration of simple LCEL
|rkharwar@nvidia.com| Author(s) | [Ruchika Kharwar](https://github.com/rasalt)

NOTE: This notebook has been tested in the following environment:
Python version = 3.10.8

## Overview

<TBD> 

## Objective
This notebook aims to show you how to leverage the retriever microservice to implement a simple LCEL.
These examples aim to be building blocks of the larger solution you will likley have in place for yout Generative AI use case.

## Before you begin
### Set up your environment.
Refer to page <> for details on how to deploy the service.
You should have docker services running in your environment thus  

docker                         dockerd                        dockerd-rootless-setuptool.sh  dockerd-rootless.sh            docker-proxy                   
nvidia@dev-h100-rkharwar-gpu01:~/retriever_03182024/docker-compose$ docker compose -f 
config/                      docker-compose-ea.yaml       docker-compose-nemollm.yaml  models/                      models_orig/                 volumes/                     
nvidia@dev-h100-rkharwar-gpu01:~/retriever_03182024/docker-compose$ docker compose -f docker-compose-ea.yaml ps
NAME                              IMAGE                                                                              COMMAND                  SERVICE          CREATED        STATUS                    PORTS
docker-compose-elasticsearch-1    docker.elastic.co/elasticsearch/elasticsearch:8.12.0                               "/bin/tini -- /usr/l…"   elasticsearch    21 hours ago   Up 21 hours (healthy)     0.0.0.0:9200->9200/tcp, :::9200->9200/tcp, 9300/tcp
docker-compose-embedding-ms-1     nvcr.io/ohlfw0olaadg/ea-participants/nemo-retriever-embedding-microservice:24.02   "/opt/nvidia/nvidia_…"   embedding-ms     21 hours ago   Up 21 hours (healthy)     
docker-compose-etcd-1             quay.io/coreos/etcd:v3.5.11                                                        "etcd -advertise-cli…"   etcd             21 hours ago   Up 21 hours (healthy)     2379-2380/tcp
docker-compose-milvus-1           milvusdb/milvus:v2.3.5                                                             "/tini -- milvus run…"   milvus           21 hours ago   Up 21 hours (healthy)     
docker-compose-minio-1            minio/minio:RELEASE.2023-03-20T20-16-18Z                                           "/usr/bin/docker-ent…"   minio            21 hours ago   Up 21 hours (healthy)     9000/tcp
docker-compose-otel-collector-1   otel/opentelemetry-collector-contrib:0.91.0                                        "/otelcol-contrib --…"   otel-collector   21 hours ago   Up 21 hours               0.0.0.0:4317->4317/tcp, :::4317->4317/tcp, 0.0.0.0:13133->13133/tcp, :::13133->13133/tcp, 0.0.0.0:55679->55679/tcp, :::55679->55679/tcp, 55678/tcp
docker-compose-postgres-1         postgres:16.1                                                                      "docker-entrypoint.s…"   postgres         21 hours ago   Up 21 hours               0.0.0.0:5432->5432/tcp, :::5432->5432/tcp
docker-compose-retrieval-ms-1     nvcr.io/ohlfw0olaadg/ea-participants/nemo-retriever-microservice:24.02             "/usr/bin/shelless_u…"   retrieval-ms     21 hours ago   Up 21 hours (unhealthy)   0.0.0.0:1984->8000/tcp, :::1984->8000/tcp
docker-compose-tika-1             apache/tika:2.9.1.0                                                                "/bin/sh -c 'exec ja…"   tika             21 hours ago   Up 21 hours               0.0.0.0:9998->9998/tcp, :::9998->9998/tcp
docker-compose-zipkin-1           openzipkin/zipkin:3.0.6                                                            "start-zipkin"           zipkin           21 hours ago   Up 21 hours (healthy)     9410/tcp, 0.0.0.0:9411->9411/tcp, :::9411->9411/tcp

### Get the NVIDIA API Catalog key
Reference: https://python.langchain.com/docs/integrations/text_embedding/nvidia_ai_endpoints

In [48]:
import getpass
import os

## API Key can be found by going to NVIDIA NGC -> AI Foundation Models -> (some model) -> Get API Code or similar.
## 10K free queries to any endpoint (which is a lot actually).

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

Valid NVIDIA_API_KEY already in environment. Delete to reset


### Setup the LLM with Nvidia Model
This could be replaced with a self hosted model

In [42]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
model = ChatNVIDIA(model="mixtral_8x7b")
#result = llm.invoke("Write a ballad about LangChain.")
#print(result.content)

## Setup the retriever client
- Initialize Client
- Create a collection
- Add a document to the collection

In [43]:
# Initialize the retriever
from nemo_retriever.retriever_client import RetrieverClient
from pprint import pprint

retriever = RetrieverClient(base_url="http://localhost:1984")

# Create a collection with the hybrid pipeline type
response = retriever.create_collection(pipeline="hybrid", name="testCollection")
collection_id = response.collection.id  # store ID of the newly created collection
print("Collection id {} created".format(collection_id))

# Add a pdf with metadata to the collection 
FILE1 = "./files/python-basics-sample-chapters.pdf"
response = retriever.add_document(collection_id=collection_id,
                                       filepath=FILE1,
                                       format="pdf",
                                       metadata={"title": "A practical introduction to python 3",
                                                 "authors": "David Amos, Dan Bader, Joanna Jablonski, Fletcher Heisler"})
    
created_document_id_1 = response.documents[0].id
print("Added file 1: {} to the collection".format(created_document_id_1))


Collection id 7bbe611e-9464-475b-84dc-9ead098e8def created
Added file 1: a5ac177509c786d782a0ae8c3224eed71d49ddaa62bc4a484388b0e0ae91cee7 to the collection


In [ ]:
## Retriever Test
- Ensure retrieval functionality works

In [44]:
# Define the query you want to search for
query = "Who are the authors of the document"

# Define the number of top results you want to retrieve
top_k = 5  # Adjust as needed

# Perform the search
search_response = retriever.search_collection(collection.id, query, top_k)

# Process and print the search results
for chunk in search_response.chunks:
    print("Chunk ID: " + chunk.id)
    print("Chunk Score: ",chunk.score)
    print("Chunk Metadata: ",chunk.metadata)
    print("-"*20)
    print("Chunk Content: ",chunk.content)
    print("-" * 80)

Chunk ID: 94c91d0995bc8d0a14d97400f3b7ba6e5f1377c8c5b5df9f3ff3fc8992dc77ac
Chunk Score:  anyof_schema_1_validator=0.9841269841269842 anyof_schema_2_validator=None actual_instance=0.9841269841269842 any_of_schemas=typing.Literal['float', 'object']
Chunk Metadata:  metadata={'title': 'A practical introduction to python 3', 'authors': 'David Amos, Dan Bader, Joanna Jablonski, Fletcher Heisler', '_indexed_at': '2024-03-21T16:06:06.926289', 'source_id': '2350732e5d1a15c9df7d70e39c948c312efbd93b858a898dc3d02893221cca4d', 'empty': 0}
--------------------
Chunk Content:  Authors
AtReal Python you’ll learn real-world programming skills from a com-munity of professional Pythonistas from all around the world.
The realpython.com website launched in 2012 and currently helpsmore than three million Python developers each month with freeprogramming tutorials and in-depth learning resources.

Everyone whoworked on this book is a practitionerwith several yearsof professional experience in the software i

## Create a simple langchain example
Reference: https://www.pinecone.io/learn/series/langchain/langchain-expression-language/
Reference: https://python.langchain.com/docs/expression_language/how_to/map

In [45]:
#from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from functools import partial

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda
)

def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))
    
def custom_retriever(query):
    # Your retrieval logic goes here, which takes the query and returns context content
    # For example, you might fetch data from a database or an API
    search_response = retriever.search_collection(collection.id, query, top_k)
    output = ""
    # Process and print the search results
    for chunk in search_response.chunks:
        output = "Chunk ID: " + str(chunk.id) + "\n"
        output = output + "Chunk Score: " + str(chunk.score) + "\n"
        output = output + "Chunk Metadata: " + str(chunk.metadata) + "\n"
        output = output + "-"*20 + "\n"
        output = output + "Chunk Content: " + str(chunk.content) + "\n"
        output = output + "-"*80
    return output


prompt_str = """Answer the question below using the context, ONLY use the context. Do no hallucinate.

Context: {context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)


retrieval_chain = (
    {"context": custom_retriever, "question": RunnablePassthrough()} |
     RPrint("Context ** ") |
     prompt |
     RPrint("Prompt ** ") |
     model |
     StrOutputParser()
)
#chain = retrieval | prompt | model | StrOutputParser
retrieval_chain.invoke("What is the book about?")

Context ** {'context': "Chunk ID: 086187b251d69289c48f58942b121a0517e667edff7a1d4442e89a167bcc7190\nChunk Score: anyof_schema_1_validator=0.4841269841269841 anyof_schema_2_validator=None actual_instance=0.4841269841269841 any_of_schemas=typing.Literal['float', 'object']\nChunk Metadata: metadata={'title': 'A practical introduction to python 3', 'authors': 'David Amos, Dan Bader, Joanna Jablonski, Fletcher Heisler', '_indexed_at': '2024-03-21T16:06:06.926289', 'source_id': '2350732e5d1a15c9df7d70e39c948c312efbd93b858a898dc3d02893221cca4d'}\n--------------------\nChunk Content: Youmight be surprised byhowmanyprofessional productsare built on Python code: Instagram, YouTube, Reddit, Spotify, toname just a few.\nPython is not only a friendly and fun language to learn, but it also pow-ers the technology behind multiple world-class companies and offersfantastic career opportunities for any programmer who masters it.\n\n1.1 Why This Book?\nLet’s face it: there’s an overwhelming amount of info

'The book is about Python programming, titled "A practical introduction to python 3". It is written for beginners who want to learn Python and are facing difficulties in figuring out what to learn and in what order to learn it. The book is written in plain English and covers the core concepts of Python in a beginner-friendly manner.'

#### Ask another question

In [46]:
#chain = retrieval | prompt | model | StrOutputParser
retrieval_chain.invoke("Who are the authors of the book?")

Context ** {'context': "Chunk ID: f4ee6055677cbd6e6e96115ac6843d398ea397dc994fa38471691fdcec2a42ec\nChunk Score: anyof_schema_1_validator=0.4841269841269841 anyof_schema_2_validator=None actual_instance=0.4841269841269841 any_of_schemas=typing.Literal['float', 'object']\nChunk Metadata: metadata={'title': 'A practical introduction to python 3', 'authors': 'David Amos, Dan Bader, Joanna Jablonski, Fletcher Heisler', '_indexed_at': '2024-03-21T16:06:06.926289', 'source_id': '2350732e5d1a15c9df7d70e39c948c312efbd93b858a898dc3d02893221cca4d'}\n--------------------\nChunk Content: Geometry Managers . . . . 57318.7 Making Your Applications Interactive . . . . . . . . 59218.8 Example App: Temperature Converter . . . . . . . . 60218.9 Example App: Text Editor . . . . . . . . . . . . . . 60718.10 Challenge: Return of the Poet . . . . . . . . . . . . . 61618.11 Summary and Additional Resources . . . . . . . . . 618\n19 Final Thoughts and Next Steps 62019.1 Free Weekly Tips for Python Developers 

"The authors of the book are David Amos, Dan Bader, Joanna Jablonski, and Fletcher Heisler. This information can be found in the Chunk Metadata, in the 'authors' field."

#### Delete the collection created in this notebook

In [47]:
try:
    retriever.delete_collection(collection_id)
    print(f"Collection '{collection.name}' deleted successfully.")
except Exception as e:
    print(f"Failed to delete collection '{collection.name}': {str(e)}")

Collection 'testCollection' deleted successfully.
